In [1]:
import pandas as pd
import sqlite3
from prettytable import PrettyTable

# 1. Load and prepare the data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"
df = pd.read_csv(url)
df = df[df['Date'].notna()]  # Remove rows with null dates

# 2. Create SQLite database and table
conn = sqlite3.connect("spacex.db")
df.to_sql("SPACEXTABLE", conn, if_exists="replace", index=False)

# 3. Create a function to run and display queries
def run_query(query):
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    
    if results:
        # Get column names
        col_names = [description[0] for description in cursor.description]
        # Create pretty table
        table = PrettyTable(col_names)
        for row in results:
            table.add_row(row)
        print(table)
    else:
        print("No results found")
    print()  # Add blank line

# 4. Now execute all tasks:

print("TASK 1: Unique launch sites")
run_query('SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE;')

print("TASK 2: 5 CCA launch sites")
run_query('SELECT * FROM SPACEXTABLE WHERE "Launch_Site" LIKE "CCA%" LIMIT 5;')

print("TASK 3: NASA CRS payload mass")
run_query('SELECT SUM("Payload_Mass__kg_") FROM SPACEXTABLE WHERE "Customer" = "NASA (CRS)";')

print("TASK 4: Avg payload for F9 v1.1")
run_query('SELECT AVG("Payload_Mass__kg_") FROM SPACEXTABLE WHERE "Booster_Version" = "F9 v1.1";')

print("TASK 5: First successful ground landing")
run_query('SELECT MIN("Date") FROM SPACEXTABLE WHERE "Landing_Outcome" = "Success (ground pad)";')

print("TASK 6: Successful drone ships (4000-6000 kg)")
run_query('''
    SELECT DISTINCT "Booster_Version" 
    FROM SPACEXTABLE 
    WHERE "Landing_Outcome" = "Success (drone ship)" 
    AND "Payload_Mass__kg_" > 4000 
    AND "Payload_Mass__kg_" < 6000;
''')

print("TASK 7: Mission outcomes count")
run_query('''
    SELECT 
        CASE 
            WHEN "Landing_Outcome" LIKE "Success%" THEN "Success"
            ELSE "Failure"
        END AS Outcome,
        COUNT(*) AS Count
    FROM SPACEXTABLE
    GROUP BY Outcome;
''')

print("TASK 8: Boosters with max payload")
run_query('''
    SELECT "Booster_Version" 
    FROM SPACEXTABLE 
    WHERE "Payload_Mass__kg_" = (
        SELECT MAX("Payload_Mass__kg_") 
        FROM SPACEXTABLE
    );
''')

print("TASK 9: 2015 drone ship failures")
run_query('''
    SELECT 
        substr("Date", 6, 2) AS Month,
        "Booster_Version",
        "Launch_Site",
        "Landing_Outcome"
    FROM SPACEXTABLE
    WHERE substr("Date", 0, 5) = "2015"
    AND "Landing_Outcome" LIKE "Failure (drone ship)%";
''')

print("TASK 10: Landing outcomes ranking (2010-2017)")
run_query('''
    SELECT 
        "Landing_Outcome",
        COUNT(*) AS OutcomeCount
    FROM SPACEXTABLE
    WHERE "Date" BETWEEN "2010-06-04" AND "2017-03-20"
    GROUP BY "Landing_Outcome"
    ORDER BY OutcomeCount DESC;
''')

# Close connection
conn.close()

TASK 1: Unique launch sites
+--------------+
| Launch_Site  |
+--------------+
| CCAFS LC-40  |
| VAFB SLC-4E  |
|  KSC LC-39A  |
| CCAFS SLC-40 |
+--------------+

TASK 2: 5 CCA launch sites
+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
|    Date    | Time (UTC) | Booster_Version | Launch_Site |                            Payload                            | PAYLOAD_MASS__KG_ |   Orbit   |     Customer    | Mission_Outcome |   Landing_Outcome   |
+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
| 2010-06-04 |  18:45:00  |  F9 v1.0  B0003 | CCAFS LC-40 |              Dragon Spacecraft Qualification Unit             |         0         |    LEO    |      Space